In [2]:
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
excel_path = r"C:\Users\NNadi\Downloads\Spectral_library_5_bucket_sand_SVC (1).xlsx"
df = pd.read_excel(excel_path)
original_wavelengths = df.iloc[:, 0].values  
signatures = df.iloc[:, 1:].values         
class_names = df.columns[1:]
num_endmembers = signatures.shape[1]
aviris_ng_wavelengths = np.arange(380, 2511, 5) 
matched_aviris_indices = []
used_indices = set()

for wl in original_wavelengths:
    diffs = np.abs(aviris_ng_wavelengths - wl)
    sorted_indices = np.argsort(diffs)
    for idx in sorted_indices:
        if idx not in used_indices:
            matched_aviris_indices.append(idx)
            used_indices.add(idx)
            break

matched_aviris_indices = sorted(matched_aviris_indices[:105])
matched_aviris_wavelengths = aviris_ng_wavelengths[matched_aviris_indices]

interpolator = interp1d(original_wavelengths, signatures, axis=0, kind='linear', bounds_error=False, fill_value="extrapolate")
signatures_resampled = interpolator(matched_aviris_wavelengths) 
num_pixels = 6000
bands = 105
output_cube = np.zeros((num_pixels, bands))
abundance_map = np.zeros((num_pixels, num_endmembers)) 
def linear_mixture(endmember_signatures, num, start_idx):
    mixed = []
    abundances = []
    for i in range(num):
        indices = np.random.choice(endmember_signatures.shape[1], 3, replace=False)
        weights = np.random.dirichlet(np.ones(3))
        pixel = np.dot(endmember_signatures[:, indices], weights)
        mixed.append(pixel)
        abundance_map[start_idx + i, indices] = weights 
    return np.array(mixed)
def nonlinear_mixture(endmember_signatures, num, start_idx):
    mixed = []
    for i in range(num):
        indices = np.random.choice(endmember_signatures.shape[1], 3, replace=False)
        weights = np.random.dirichlet(np.ones(3))
        base = np.dot(endmember_signatures[:, indices], weights)
        interaction = endmember_signatures[:, indices[0]] * endmember_signatures[:, indices[1]]
        pixel = base + 0.2 * interaction
        mixed.append(np.clip(pixel, 0, 1))
        abundance_map[start_idx + i, indices] = weights
    return np.array(mixed)
def hapke_mixture(endmember_signatures, num, start_idx):
    mixed = []
    for i in range(num):
        indices = np.random.choice(endmember_signatures.shape[1], 2, replace=False)
        w = np.random.uniform(0.3, 0.7)
        r1 = endmember_signatures[:, indices[0]]
        r2 = endmember_signatures[:, indices[1]]
        pixel = (w * r1) / (1 + r1) + ((1 - w) * r2) / (1 + r2)
        mixed.append(np.clip(pixel, 0, 1))
        abundance_map[start_idx + i, indices[0]] = w
        abundance_map[start_idx + i, indices[1]] = 1 - w
    return np.array(mixed)
def bilinear_mixture(endmember_signatures, num, start_idx):
    mixed = []
    for i in range(num):
        indices = np.random.choice(endmember_signatures.shape[1], 2, replace=False)
        weights = np.random.dirichlet(np.ones(2))
        r1 = endmember_signatures[:, indices[0]]
        r2 = endmember_signatures[:, indices[1]]
        interaction = r1 * r2
        pixel = weights[0] * r1 + weights[1] * r2 + 0.1 * interaction
        mixed.append(np.clip(pixel, 0, 1))
        abundance_map[start_idx + i, indices[0]] = weights[0]
        abundance_map[start_idx + i, indices[1]] = weights[1]
    return np.array(mixed)
def ppnm_mixture(endmember_signatures, num, start_idx):
    mixed = []
    for i in range(num):
        indices = np.random.choice(endmember_signatures.shape[1], 3, replace=False)
        weights = np.random.dirichlet(np.ones(3))
        linear_mix = np.dot(endmember_signatures[:, indices], weights)
        pixel = linear_mix + 0.05 * (linear_mix ** 2)
        mixed.append(np.clip(pixel, 0, 1))
        abundance_map[start_idx + i, indices] = weights
    return np.array(mixed)
def multiscattering_mixture(endmember_signatures, num, start_idx):
    mixed = []
    for i in range(num):
        indices = np.random.choice(endmember_signatures.shape[1], 3, replace=False)
        weights = np.random.dirichlet(np.ones(3))
        r = endmember_signatures[:, indices]
        base = np.dot(r, weights)
        scatter = np.mean([r[:, j] * r[:, (j+1)%3] for j in range(3)], axis=0)
        pixel = base + 0.1 * scatter
        mixed.append(np.clip(pixel, 0, 1))
        abundance_map[start_idx + i, indices] = weights
    return np.array(mixed)
def intimate_mixture(endmember_signatures, num, start_idx):
    mixed = []
    for i in range(num):
        indices = np.random.choice(endmember_signatures.shape[1], 2, replace=False)
        weights = np.random.dirichlet(np.ones(2))
        r1 = endmember_signatures[:, indices[0]]
        r2 = endmember_signatures[:, indices[1]]
        pixel = 1 - weights[0]*(1 - r1) * (1 - r2)
        mixed.append(np.clip(pixel, 0, 1))
        abundance_map[start_idx + i, indices] = weights
    return np.array(mixed)

output_cube[0:1000] = linear_mixture(signatures_resampled, 1000, 0)
output_cube[1000:2000] = nonlinear_mixture(signatures_resampled, 1000, 1000)
output_cube[2000:3000] = bilinear_mixture(signatures_resampled, 1000, 2000)
output_cube[3000:4000] = ppnm_mixture(signatures_resampled, 1000, 3000)
output_cube[4000:5000] = multiscattering_mixture(signatures_resampled, 1000, 4000)
output_cube[5000:] = intimate_mixture(signatures_resampled, 1000, 5000)


# Add small Gaussian noise
output_cube += np.random.normal(0, 0.005, output_cube.shape)
output_cube = np.clip(output_cube, 0, 1)

labels = np.array([0]*1000 + [1]*1000 + [2]*1000 + [3]*1000 + [4]*1000 + [5]*1000)


# Save results
print(output_cube.shape)   # shape (6000, 105)
print(labels.shape)      # shape (6000,)
print(matched_aviris_wavelengths.shape)  # shape (105,)
print( abundance_map.shape)  # shape (6000, num_endmembers)


(6000, 105)
(6000,)
(105,)
(6000, 9)


In [3]:
print(signatures_resampled.shape)

(105, 9)


In [4]:
import numpy as np
import pandas as pd
from scipy.optimize import nnls, minimize
from sklearn.linear_model import Lasso, ElasticNet
from sklearn.kernel_ridge import KernelRidge
from sklearn.decomposition import NMF
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.spatial.distance import cosine
from sklearn.linear_model import Ridge
# Load synthetic data
X = output_cube 
gt_abundances = abundance_map  
M = signatures_resampled  

def spectral_angle(true, pred):
    return np.arccos(np.clip(np.dot(true, pred) / (np.linalg.norm(true) * np.linalg.norm(pred) + 1e-10), -1, 1))

def evaluate(abundances_pred, abundances_true, name=""):
    rmse = np.sqrt(np.mean((abundances_pred - abundances_true) ** 2))
    mae = mean_absolute_error(abundances_true, abundances_pred)
    sam = np.mean([spectral_angle(gt, est) for gt, est in zip(abundances_true, abundances_pred)])
    return {"Method": name, "RMSE": rmse, "MAE": mae, "SAM (deg)": np.degrees(sam)}

# --- Unmixing Methods ---

def fcls_unmixing(X, M):
    A_hat = np.zeros((X.shape[0], M.shape[1]))
    for i, x in enumerate(X):
        def objective(a):
            return np.linalg.norm(x - M @ a) ** 2
        cons = ({'type': 'eq', 'fun': lambda a: np.sum(a) - 1},
                {'type': 'ineq', 'fun': lambda a: a})
        res = minimize(objective, x0=np.ones(M.shape[1]) / M.shape[1], constraints=cons, method='SLSQP')
        A_hat[i] = res.x
    return np.clip(A_hat, 0, 1)

def ucls_unmixing(X, M):
    A_hat = np.linalg.pinv(M) @ X.T
    return A_hat.T

def nnls_unmixing(X, M):
    A_hat = np.zeros((X.shape[0], M.shape[1]))
    for i in range(X.shape[0]):
        A_hat[i], _ = nnls(M, X[i])
    return A_hat

def lasso_unmixing(X, M, alpha=0.001):
    A_hat = np.zeros((X.shape[0], M.shape[1]))
    for i in range(X.shape[0]):
        model = Lasso(alpha=alpha, positive=True, fit_intercept=False, max_iter=10000)
        model.fit(M, X[i])
        A_hat[i] = model.coef_
    return A_hat

def elasticnet_unmixing(X, M, alpha=0.01, l1_ratio=0.5):
    A_hat = np.zeros((X.shape[0], M.shape[1]))
    for i in range(X.shape[0]):
        model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, positive=True, fit_intercept=False, max_iter=10000)
        model.fit(M, X[i])
        A_hat[i] = model.coef_
    return A_hat

def khype_unmixing_fixed(X, M, alpha=0.01):
    """
    Approximate KHype using pixel-wise Ridge regression.
    X: (n_pixels, n_bands)
    M: (n_bands, n_endmembers)
    Returns: A_hat (n_pixels, n_endmembers)
    """
    A_hat = np.zeros((X.shape[0], M.shape[1]))

    for i in range(X.shape[0]):
        model = Ridge(alpha=alpha, fit_intercept=False)
        model.fit(M, X[i])  
        A_hat[i] = model.coef_
    
    return np.clip(A_hat, 0, 1)

def nmf_unmixing(X, M, max_iter=1000):
    """
    NMF-based spectral unmixing.
    X: (n_pixels, n_bands)
    M is only used to get number of endmembers
    """
    n_components = M.shape[1]
    model = NMF(n_components=n_components, init='nndsvda', max_iter=max_iter, random_state=42)
    A_hat = model.fit_transform(X)
    return np.clip(A_hat, 0, 1)


def naive_pinv_unmixing(X, M):
    return np.dot(X, np.linalg.pinv(M.T))

results = []
methods = {
    "FCLS": fcls_unmixing,
    "UCLS": ucls_unmixing,
    "NNLS": nnls_unmixing,
    "LASSO": lasso_unmixing,
    "ElasticNet": elasticnet_unmixing,
    "KHype": khype_unmixing_fixed,
    "NMF": nmf_unmixing,
    "Pseudo-Inverse": naive_pinv_unmixing
}

for name, method in methods.items():
    try:
        A_hat = method(X, M)
        metrics = evaluate(A_hat, gt_abundances, name)
        print(metrics)
        results.append(metrics)
    except Exception as e:
        results.append({"Method": name, "RMSE": None, "MAE": None, "SAM (deg)": None})



{'Method': 'FCLS', 'RMSE': 0.17690837016276445, 'MAE': 0.06196762165206567, 'SAM (deg)': 26.315390198494963}
{'Method': 'UCLS', 'RMSE': 1.2977673536571375, 'MAE': 0.40957362061114805, 'SAM (deg)': 44.10489910866771}
{'Method': 'NNLS', 'RMSE': 0.20585408083321677, 'MAE': 0.0703204522118231, 'SAM (deg)': 20.829680521847187}
{'Method': 'LASSO', 'RMSE': 0.25769297581457745, 'MAE': 0.12537174463777548, 'SAM (deg)': 50.890176840520255}
{'Method': 'ElasticNet', 'RMSE': 0.2551538085888919, 'MAE': 0.14484561813376057, 'SAM (deg)': 64.09175252526782}
{'Method': 'KHype', 'RMSE': 0.22167812075234117, 'MAE': 0.12162619345190435, 'SAM (deg)': 33.23241782204086}
{'Method': 'NMF', 'RMSE': 0.24443022612430348, 'MAE': 0.17303584421681295, 'SAM (deg)': 65.30095305259688}
{'Method': 'Pseudo-Inverse', 'RMSE': 1.2977673536571859, 'MAE': 0.4095736206111597, 'SAM (deg)': 44.10489910866787}
